In [29]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import json
from time import sleep

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
caps = DesiredCapabilities.CHROME
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
driver = webdriver.Chrome(options=options)
driver.get('http://google.com')
def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

browser_log = driver.get_log('performance') 
# print(browser_log)
events = [process_browser_log_entry(entry) for entry in browser_log]
events = [event for event in events if 'Network.response' in event['method']]
print(events)

driver.quit()

In [37]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import json
from time import sleep
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
caps = DesiredCapabilities.CHROME
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
driver = webdriver.Chrome(options=options)
driver.get('http://google.com')
# sleep(5)  # wait for the requests to take place

# extract requests from logs
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

def log_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
    )

for log in filter(log_filter, logs):
    request_id = log["params"]["requestId"]
    resp_url = log["params"]["response"]["url"]
    if "async" in resp_url:
        print(f"Caught {resp_url}\n")
        print(driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id}), "\n")

Caught https://ogads-pa.googleapis.com/$rpc/google.internal.onegoogle.asyncdata.v1.AsyncDataService/GetAsyncData

{'base64Encoded': False, 'body': '[null,null,null,null,null,null,null,null,null,null,0]'} 

